# Advent of Code 2022

In [1]:
YEAR=2022

# Helpers


These helper functions are useful to answer the questions. These (and using a Jupyter notebook in the first place) are inspired by Peter Norvig

## Robustness

In [2]:
# Allows for refactoring after having determined the answer.
def assert_eq(expected, got, show):
    if expected == got:
        return expected
    else:
        raise Exception(f"Expected '{expected}', but got '{got}'")

def assert_example(expected, got):
    assert_eq(expected, got, show=False)

def assert_answer(name, expected, got):
    print("Answer for", name, "is", assert_eq(expected, got, show=True))

## Input processing

In [3]:
def read_input(name):
    with open(f"inputs/{YEAR}/{name}") as f:
        return f.read().strip()
    
# Parse a line-based input file, where each line is pared by function passed to `line_parser` 
def parse_input(name, line_parser=str, preview_lines=10):
    lines = tmap(line_parser, read_input(name).split("\n"))
    print("Lines:", len(lines))
    print(f"First {preview_lines} lines:")
    for preview_line in lines[0:preview_lines]:
      print(" ", preview_line)
    return lines

# Splits a line on a separator, and then either apply the one function, or list of functions to the parts.
def parse_split(sep=" ", fns=str):
    def inner(line):
        parts = line.split(sep)
        if isinstance(fns, list):
            return [fn(part) for (fn,part) in zip(fns, parts)]
        else:
            return [fns(part) for part in parts]
    return inner


## Strict functions

In [4]:
# By default, `map` and `filter` are lazy. Not always practical in a notebook where you might want to inspect the data :)
def tmap(fn, *args):
    return tuple(map(fn, *args))
def tfilter(fn, *args):
    return tuple(filter(fn, *args))

## Data analysis

In [5]:
def count_pred_true(pred, data):
    return sum([1 for item in data if pred(item)])

# Return a sliding window (filled) of size `ws`
def windowed(data, ws):
    return [data[i:i+ws] for i in range(0, len(data)-ws+1)]

# Day 1: Calorie Counting

## Puzzle 1

In [25]:
raw_example_input01= """
1000
2000
3000

4000

5000
6000

7000
8000
9000

10000
""".strip()

def parse_input01(txt):
    return sorted(tmap(lambda t: sum(tmap(int, t.split("\n"))), txt.strip().split("\n\n")), reverse=True)

def puzzle_01a(elves):
    return elves[0]
            
example_input01 = parse_input01(raw_example_input01)

assert_example(24000, puzzle_01a(example_input01))

In [27]:
input01 = parse_input01(read_input("01"))

assert_answer("1a", 67658, puzzle_01a(input01))

Answer for 1a is 67658


## Puzzle 2

In [32]:
def puzzle_01b(elves):
    return sum(elves[0:3])

assert_answer("1b", 200158, puzzle_01b(input01))


Answer for 1b is 200158
